In [63]:
#libraries
import pyodbc #connecting to SQL
import pandas as pd #data analysis
import matplotlib.pyplot as plt #plots
import numpy as np #numerical manipulations
import matplotlib.patches as mpatches #for patches for a legend

#connecting to the database
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=CHRIS-PC\SERVER1;' #user name and server name
                      'Database=AdventureWorks2019;'
                      'Trusted_Connection=yes;')

In [64]:
cursor = conn.cursor() #creating a cursor to execute SQL queries

#executing query copied from SSMS
cursor.execute("""
    SELECT 
        Title_Group,
        AVG(avg_slh_pt) AS AVG_SickLeaveHours, 
        CASE 
            WHEN Title_Group = 'Chiefs' THEN AVG(CASE WHEN Level != 0 THEN Level ELSE NULL END)
            ELSE AVG(Level)
        END AS Avg_Level
    FROM (
        SELECT 
            CASE 
                WHEN JobTitle LIKE '%Manager%' THEN 'Managers'
                WHEN JobTitle LIKE '%Assistant%' THEN 'Assistants'
                WHEN JobTitle LIKE '%Specialist%' THEN 'Specialists'
                WHEN JobTitle LIKE '%Chief%' AND JobTitle NOT LIKE '%Assistant%' THEN 'Chiefs'
                WHEN JobTitle LIKE '%Vice President%' THEN 'Vice Presidents'
                WHEN JobTitle LIKE '%Supervisor%' THEN 'Supervisors'
                WHEN JobTitle LIKE '%Technician%' THEN 'Technicians'
                WHEN JobTitle LIKE '%Engineer%' THEN 'Engineers'
                WHEN JobTitle LIKE '%Designer%' THEN 'Designers'
                WHEN JobTitle LIKE '%Administrator%' THEN 'Administrators'
                WHEN JobTitle LIKE '%Scheduler%' THEN 'Schedulers'
                WHEN JobTitle LIKE '%Buyer%' THEN 'Buyers'
                WHEN JobTitle LIKE '%Janitor%' THEN 'Janitors'
                WHEN JobTitle LIKE '%Stocker%' THEN 'Stockers'
                WHEN JobTitle LIKE '%Clerk%' THEN 'Clerks'
                WHEN JobTitle LIKE '%Accountant%' THEN 'Accountants'
                WHEN JobTitle LIKE '%Recruiter%' THEN 'Recruiters'
                WHEN JobTitle LIKE '%Representative%' THEN 'Representatives'
                ELSE 'Other Titles'
            END AS Title_Group, 
            ISNULL(OrganizationLevel, 0) AS Level,
            AVG(SickLeaveHours) AS avg_slh_pt
        FROM 
            HumanResources.Employee
        GROUP BY 
            JobTitle, 
            OrganizationLevel
    ) AS subquery
    GROUP BY 
        Title_Group
    ORDER BY 
        Avg_Level DESC;
""")

rows = cursor.fetchall() #to return the whole result from the query

conn.close() #closing the connection

In [65]:
#extracting groups, avg sick leave hours and avg levels from the result of the query
title_groups = [row[0] for row in rows]
avg_sick_leave_hours = [row[1] for row in rows]
avg_level = [row[2] for row in rows]

In [66]:
#now sorting based on levels in DESC 
sorted_idx = np.argsort(avg_level)[::-1] 
title_groups_sorted = [title_groups[i] for i in sorted_idx]
avg_sick_leave_hours_sorted = [avg_sick_leave_hours[i] for i in sorted_idx]
avg_level_sorted = [avg_level[i] for i in sorted_idx]

In [67]:
#i want different colors for different levels
level_colors = {
    1: 'red',
    2: 'orange',
    3: 'yellow',
    4: 'green'
}

#changing to a different style
plt.style.use('dark_background')

In [68]:
#creating a plot
#i want job title groups on x axis and avg sick ave hours on y axis
#also using defined colors from the code above for different levels
fig, ax = plt.subplots()
x = np.arange(len(title_groups_sorted))
width = 0.70

bars = ax.bar(x, avg_sick_leave_hours_sorted, width, label='Avg Sick Leave Hours', color=[level_colors[level] for level in avg_level_sorted])

#adding a midpoint that represents avg sick leave hours
midpoint = np.mean(avg_sick_leave_hours_sorted)
ax.axhline(midpoint, color='white', linestyle='--', label='Midpoint')

#setting ticks/labels/title
ax.set_xticks(x)
ax.set_xticklabels([f'{title}' for title in title_groups_sorted], rotation=45, ha='right')
ax.set_xlabel('Job Title Groups')
ax.set_ylabel('Sick Leave Hours')
ax.set_title('Average Sick Leave Hours by Title Groups')

#adding a legend
legend_patches = [mpatches.Patch(color=color, label=f'Level {level}') for level, color in level_colors.items()]
legend_patches.append(mpatches.Patch(color='white', label='Midpoint'))
ax.legend(handles=legend_patches, loc='upper left')

#adjusting the layout for a better picture; displaying the plot
plt.tight_layout()
plt.show()

SyntaxError: unterminated string literal (detected at line 30) (1358657032.py, line 30)